In [19]:
import pandas as pd


train=pd.read_csv("electric_train.csv")


# drop first column
train.drop(train.columns[0], axis=1, inplace=True)

In [20]:
test=pd.read_csv("electric_test.csv")
# drop first column
test.drop(test.columns[0], axis=1, inplace=True)

In [21]:
train.columns
for idx, col in enumerate(train.columns):
    print(idx, col)
    train.rename(columns={col: col.replace("electric_train.", "")}, inplace=True)

0 electric_train.num
1 electric_train.tm
2 electric_train.hh24
3 electric_train.n
4 electric_train.stn
5 electric_train.sum_qctr
6 electric_train.sum_load
7 electric_train.n_mean_load
8 electric_train.nph_ta
9 electric_train.nph_hm
10 electric_train.nph_ws_10m
11 electric_train.nph_rn_60m
12 electric_train.nph_ta_chi
13 electric_train.weekday
14 electric_train.week_name
15 electric_train.elec


In [22]:
train.columns

Index(['num', 'tm', 'hh24', 'n', 'stn', 'sum_qctr', 'sum_load', 'n_mean_load',
       'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi', 'weekday',
       'week_name', 'elec'],
      dtype='object')

In [23]:
# split datetime tm into date and time
train['hour'] = pd.to_datetime(train['tm']).dt.hour   
train['month'] = pd.to_datetime(train['tm']).dt.month
train['day'] = pd.to_datetime(train['tm']).dt.day

In [24]:
train.drop('tm', axis=1, inplace=True)

In [25]:
train

,num,hh24,n,stn,sum_qctr,sum_load,n_mean_load,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elec,hour,month,day
0,4821,1,11,884,6950,751.32,68.606449,2.2,62.7,1.8,0.0,-1.0,4,0,99.56,1,1,1
1,4821,2,11,884,6950,692.60,68.606449,2.3,63.1,2.1,0.0,-0.6,4,0,91.78,2,1,1
2,4821,3,11,884,6950,597.48,68.606449,2.2,62.4,2.5,0.0,-1.3,4,0,79.17,3,1,1
3,4821,4,11,884,6950,553.48,68.606449,1.7,63.5,1.7,0.0,-0.2,4,0,73.34,4,1,1
4,4821,5,11,884,6950,526.24,68.606449,1.7,63.0,1.6,0.0,-0.8,4,0,69.73,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593350,20947,20,23,671,34200,6779.84,225.462000,2.7,46.3,3.1,0.0,-0.4,5,1,130.74,20,12,31
7593351,20947,21,23,671,34200,6802.40,225.462000,2.6,46.8,3.1,0.0,-0.5,5,1,131.18,21,12,31
7593352,20947,22,23,671,34200,6706.68,225.462000,2.4,47.4,2.1,0.0,0.2,5,1,129.33,22,12,31
7593353,20947,23,23,671,34200,6355.88,225.462000,2.5,47.0,2.1,0.0,0.3,5,1,122.57,23,12,31


In [26]:
from pycaret.regression import *


In [27]:
train.drop(columns=['sum_qctr','sum_load','n_mean_load'], inplace=True)


In [28]:
from autogluon.tabular import TabularDataset, TabularPredictor

#train = pd.read_csv('../preprocess/q1_train_data_2.csv')
#test = pd.read_csv('../preprocess/q1_test_data_2.csv')

train_data = TabularDataset(train)
#test_data = TabularDataset(test)

print("==================Tabular_complete========================")

hyperparameters = {
	'GBM': [{}, {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, 'GBMLarge'],
	'CAT': {},
	'XGB': {},
}

save_path = './models'
predictor = TabularPredictor(label='elec', problem_type = 'regression', eval_metric='mean_squared_error',
                            path = save_path)
predictor.fit(train_data, hyperparameters=hyperparameters,presets='medium_quality')

print("==================learning_complete========================")

Presets specified: ['medium_quality']
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "./models"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          12
Memory Avail:       3.26 GB / 15.93 GB (20.5%)
Disk Space Avail:   34.99 GB / 237.70 GB (14.7%)
Train Data Rows:    7593355
Train Data Columns: 14
Label Column:       elec
Problem Type:       regression
Preprocessing data ...


==================Tabular_complete========================


Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    3247.94 MB
	Train Data (Original)  Memory Usage: 724.16 MB (22.3% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 5 | ['nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi']
		('int', [])   : 9 | ['num', 'hh24', 'n', 'stn', 'weekday', ...]
	Types of features in processed data (raw d

[1000]	valid_set's l2: 23.7281
[2000]	valid_set's l2: 18.4396
[3000]	valid_set's l2: 15.8905
[4000]	valid_set's l2: 13.8829
[5000]	valid_set's l2: 12.5966
[6000]	valid_set's l2: 11.52
[7000]	valid_set's l2: 10.6415
[8000]	valid_set's l2: 9.94
[9000]	valid_set's l2: 9.39342
[10000]	valid_set's l2: 8.89453


	-8.8945	 = Validation score   (-mean_squared_error)
	1011.95s	 = Training   runtime
	3.52s	 = Validation runtime
Fitting model: LightGBMXT ...
	To force training the model, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=1.26 to avoid the error)
		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max_memory_usage_ratio": VALUE})`
		Setting "ag.max_memory_usage_ratio" to values above 1 may result in out-of-memory errors. You may consider using a machine with more memory as a safer alternative.
	Not enough memory to train LightGBMXT... Skipping this model.
Fitting model: CatBoost ...
	To force training the model, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=1.11 to avoid the error)
		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max

==================learning_complete========================


In [29]:
predictor.leaderboard(train_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,-8.716142,-8.894532,mean_squared_error,339.398070,3.520025,1011.952992,339.398070,3.520025,1011.952992,1,True,1
1,WeightedEnsemble_L2,-8.716142,-8.894532,mean_squared_error,339.528188,3.521027,1011.964422,0.130118,0.001002,0.011430,2,True,2


In [35]:
test['hour'] = pd.to_datetime(test['tm']).dt.hour
test['month'] = pd.to_datetime(test['tm']).dt.month
test['day'] = pd.to_datetime(test['tm']).dt.day
test['n']=0

In [36]:
test.drop('tm', axis=1, inplace=True)

In [34]:
train

,num,hh24,n,stn,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elec,hour,month,day
0,4821,1,11,884,2.2,62.7,1.8,0.0,-1.0,4,0,99.56,1,1,1
1,4821,2,11,884,2.3,63.1,2.1,0.0,-0.6,4,0,91.78,2,1,1
2,4821,3,11,884,2.2,62.4,2.5,0.0,-1.3,4,0,79.17,3,1,1
3,4821,4,11,884,1.7,63.5,1.7,0.0,-0.2,4,0,73.34,4,1,1
4,4821,5,11,884,1.7,63.0,1.6,0.0,-0.8,4,0,69.73,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593350,20947,20,23,671,2.7,46.3,3.1,0.0,-0.4,5,1,130.74,20,12,31
7593351,20947,21,23,671,2.6,46.8,3.1,0.0,-0.5,5,1,131.18,21,12,31
7593352,20947,22,23,671,2.4,47.4,2.1,0.0,0.2,5,1,129.33,22,12,31
7593353,20947,23,23,671,2.5,47.0,2.1,0.0,0.3,5,1,122.57,23,12,31


In [37]:
test_data = TabularDataset(test)
y_pred = predictor.predict(test_data)

In [39]:
y_pred.to_csv('electric_result.csv')

In [30]:
for idx, col in enumerate(test.columns):
    print(idx, col)
    test.rename(columns={col: col.replace("electric_test.", "")}, inplace=True)



0 electric_test.num
1 electric_test.tm
2 electric_test.hh24
3 electric_test.stn
4 electric_test.nph_ta
5 electric_test.nph_hm
6 electric_test.nph_ws_10m
7 electric_test.nph_rn_60m
8 electric_test.nph_ta_chi
9 electric_test.weekday
10 electric_test.week_name


In [31]:
test.columns

Index(['num', 'tm', 'hh24', 'stn', 'nph_ta', 'nph_hm', 'nph_ws_10m',
       'nph_rn_60m', 'nph_ta_chi', 'weekday', 'week_name'],
      dtype='object')